# IMPORT & SET UP

In [1]:
%load_ext autotime
import pandas as pd
import numpy as np
import time
from datetime import datetime
import urllib.parse
import os
import re
import json

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.common.exceptions import NoSuchElementException

pd.set_option('display.max_colwidth', None)

time: 396 ms (started: 2023-03-31 20:15:12 -05:00)


In [13]:
PROJECT_DIR = '/Users/minhnguyen/Desktop/Minh Nguyen/Projects/linkedin_analysis'
ID_DIR_NAME = 'job_id_dir'
INFO_DIR_NAME = 'job_info_dir'

time: 220 µs (started: 2023-03-31 20:19:43 -05:00)


### DRIVER FUNCTIONS

In [3]:
DRIVER = None

time: 166 µs (started: 2023-03-31 20:15:16 -05:00)


In [4]:
def start_driver():
    global DRIVER
    if DRIVER is None:
        options = Options()
        options.add_argument('--headless=new')
        options.add_argument('--no-sandbox')
        options.add_argument('--incognito')
        options.add_argument('-disable-dev-shm-usage')

        chromedriver_path = PROJECT_DIR + '/chromedriver_mac_arm64/chromedriver'
        service = Service(chromedriver_path)
        DRIVER = webdriver.Chrome(service=service, options=options)
    else:
        print('A driver is already running. Please close current driver before starting another one.')

def close_driver():
    global DRIVER
    if DRIVER is not None:
        DRIVER.close()
        DRIVER = None

time: 419 µs (started: 2023-03-31 20:15:16 -05:00)


In [5]:
close_driver()

time: 148 µs (started: 2023-03-31 20:15:17 -05:00)


In [6]:
start_driver()

time: 1.52 s (started: 2023-03-31 20:15:18 -05:00)


### URL & LOGIN SETUPS

In [7]:
from linkedin_scraper import actions

EMAIL = "minhnnt97@gmail.com"
PASSWORD = "pazqig-higdar-nehmE6"
actions.login(DRIVER, EMAIL, PASSWORD) 

BASE_URL = "https://www.linkedin.com/jobs/search/"
SEARCH_TERM = "Data Scientist"
LOCATION_TERM = "United States"

SEARCH_URL = BASE_URL + f"?keywords={urllib.parse.quote(SEARCH_TERM)}" + f"&location={urllib.parse.quote(LOCATION_TERM)}&refresh=true"
SEARCH_URL

'https://www.linkedin.com/jobs/search/?keywords=Data%20Scientist&location=United%20States&refresh=true'

time: 8.33 s (started: 2023-03-31 20:15:27 -05:00)


# FUNCTIONS

## Function to scrape all job IDs on the search page(s)

In [8]:
# Function to scrape all products from a page
def get_all_job_ids_from_page(url, num_page=-1):

    global DRIVER, SEARCH_TERM, LOCATION_TERM

    job_id_scrape_time = datetime.now()
    print(f'DATE & TIME: {job_id_scrape_time.strftime(("%Y/%m/%d %H:%M:%S"))}')
    job_id_list = []      
    DRIVER.get(url) 
    time.sleep(5) 

      # Count total number of pages in the search result
    max_page = DRIVER.find_element(By.CLASS_NAME, 'jobs-search-results-list__pagination').find_element(By.TAG_NAME, 'ul').find_elements(By.TAG_NAME, 'li')[-1]
    max_page = int(max_page.text)
    num_page = min(max_page, num_page) if num_page>0 else max_page
    print(f'Scraping {num_page} page(s) out of {max_page} total pages for {SEARCH_TERM} jobs search in {LOCATION_TERM}...')
    total_err_count = 0

      # Loop through num_page pages to scrape all products listed on each page
    for p in range(1, num_page+1):
      url = url + f'&start={(p-1)*25}'  # p=1: start=0, p=2: start = 25 ...
      DRIVER.get(url)
      time.sleep(3)
      # Find all li tags that contain the job information
      products_all = DRIVER.find_element(By.CLASS_NAME, "jobs-search-results-list").find_element(By.CLASS_NAME, "scaffold-layout__list-container").find_elements(By.CLASS_NAME, "scaffold-layout__list-item")
      print(f'Found {len(products_all)} job IDs on page {p}.', end=' ')

      err_count = 0
      for product in products_all:
          try:
              job_id = product.get_attribute("data-occludable-job-id")
              job_id_list.append(job_id)
          except NoSuchElementException:
              err_count += 1

      print(f'Could not scrape {err_count} jobs.')
      total_err_count += err_count

    # Remove dupes
    job_id_list = list(set(job_id_list))
    print(f'---> Found total {len(job_id_list)} unique jobs. Could not scrape total of {total_err_count} jobs.')

    return job_id_list, job_id_scrape_time


time: 1.21 ms (started: 2023-03-31 20:15:35 -05:00)


## Function to scrape information on the job page

In [11]:
def get_single_job_info(job_id):

    info = {
        'Job ID': job_id,
        'Job URL': None,
        'Name': None,
        'Company': None,
        'Company Logo URL': None,
        'Location': None,
        'Workplace Type': None,
        'Time Posted': None,
        'Applicants Count': None,
        'Job Overview': None,
        'Company Overview': None,
        'HR URL': None
    }

    JOB_URL = f"https://www.linkedin.com/jobs/view/{job_id}"
    DRIVER.get(JOB_URL)
    time.sleep(3)

    # Job URL
    info['Job URL'] = JOB_URL

    # Job Name
    try:
        name = DRIVER.find_element(By.CLASS_NAME, "jobs-unified-top-card__job-title").get_attribute("innerHTML")
        info['Name'] = name.strip()
    except NoSuchElementException:
        pass

    # Company
    try:
        company = DRIVER.find_element(By.CLASS_NAME, "jobs-unified-top-card__company-name").find_element(By.TAG_NAME, 'a').get_attribute("innerHTML")
        info['Company'] = company.strip()
    except NoSuchElementException:
        pass

    # Company Logo
    try:
        comp_logo_url = DRIVER.find_element(By.CLASS_NAME, 'p5').find_element(By.TAG_NAME, 'img').get_attribute('src')
        info['Company Logo URL'] = comp_logo_url.strip()
    except NoSuchElementException:
        pass

    # Location
    try:
        location = DRIVER.find_element(By.CLASS_NAME, "jobs-unified-top-card__subtitle-primary-grouping").find_element(By.CLASS_NAME, "jobs-unified-top-card__bullet").get_attribute("innerHTML")
        info['Location'] = location.strip()
    except NoSuchElementException:
        pass

    # Workplace Type
    try:
        work_type = DRIVER.find_element(By.CLASS_NAME, "jobs-unified-top-card__workplace-type").get_attribute("innerHTML")
        info['Workplace Type'] = work_type.strip()
    except NoSuchElementException:
        pass

    # Time Posted
    try:
        time_posted = DRIVER.find_element(By.CLASS_NAME, "jobs-unified-top-card__posted-date").get_attribute("innerHTML")
        info['Time Posted'] = time_posted.strip()
    except NoSuchElementException:
        pass

    # Applicants Count
    try:
        applicants = DRIVER.find_element(By.CLASS_NAME, "jobs-unified-top-card__subtitle-secondary-grouping").find_element(By.CLASS_NAME, "jobs-unified-top-card__applicant-count").get_attribute("innerHTML")
        info['Applicants Count'] = applicants.strip()
    except NoSuchElementException:
        try:
            applicants = DRIVER.find_element(By.CLASS_NAME, "jobs-unified-top-card__subtitle-secondary-grouping").find_element(By.CLASS_NAME, "jobs-unified-top-card__bullet").get_attribute("innerHTML")
            info['Applicants Count'] = applicants.strip()
        except NoSuchElementException:
            pass

    # Job & Company Insight
    try:
        job_overview, company_overview = DRIVER.find_elements(By.CLASS_NAME, "jobs-unified-top-card__job-insight")[:2]
        # use try except to find info of each column:
        # try:
        #     salary = job_overview.find_element(By.TAG_NAME, 'span').find_element(By.TAG_NAME, 'a').get_attribute("innerHTML")
        #     info['Salary'] = salary.strip('\n<!-> ')
        #     job_overview = job_overview.find_element(By.TAG_NAME, 'span').text # With salary posted
        # except NoSuchElementException:
        #     job_overview = job_overview.find_element(By.TAG_NAME, 'span').get_attribute("innerHTML") # Without salary posted

        # use .text to get all text information
        job_overview = job_overview.find_element(By.TAG_NAME, 'span').text
        job_overview = re.sub(r'<!--(?=.*?-->).*?-->', '', job_overview, flags=re.DOTALL) # Remove all HTML comments

        company_overview = company_overview.find_element(By.TAG_NAME, 'span').get_attribute("innerHTML")
        company_overview = re.sub(r'<!--(?=.*?-->).*?-->', '', company_overview, flags=re.DOTALL) # Remove all HTML comments

        info['Job Overview'] = job_overview.strip()
        info['Company Overview'] = company_overview.strip()
    except NoSuchElementException:
        pass

    # HR URL
    try:
        hr_url = DRIVER.find_element(By.CSS_SELECTOR, "div[class*='hirer-card__hirer-information'] a").get_attribute('href')
        info['HR URL'] = hr_url.strip()
    except NoSuchElementException:
        pass

        # Job details
    # try:
    #     job_details = DRIVER.find_element(By.ID, 'job-details').find_element(By.TAG_NAME, 'span').get_attribute('innerHTML')
    #     job_details = re.sub(r'<!--(?=.*?-->).*?-->', '', job_details, flags=re.DOTALL) # Remove all HTML comments
    #     info['Job Details'] = job_details.strip()
    # except NoSuchElementException:
    #     pass

    return info


time: 1.68 ms (started: 2023-03-31 20:16:12 -05:00)


#### Test job_id
- 3437480788
- 3525179725
- 3540119207
- 3515314825

In [10]:
test_info = get_single_job_info(3525179725) # apps count
test_info

{'Job ID': 3525179725,
 'Job URL': 'https://www.linkedin.com/jobs/view/3525179725',
 'Name': 'AI Research Engineer- Remote',
 'Company': 'Neo Cybernetica',
 'Company Logo URL': 'https://media.licdn.com/dms/image/C560BAQFdt_fuOIJ5TQ/company-logo_100_100/0/1644369586489?e=1688601600&v=beta&t=QyxXmGwyZyxqfDP7SiKcGuUJGrnmgnsnpdG5FbV-ESw',
 'Location': 'Bedford, NH',
 'Workplace Type': 'Remote',
 'Time Posted': '2 weeks ago',
 'Applicants Count': '76 applicants',
 'Job Overview': '11-50 employees · Software Development',
 'Company Overview': 'See how you compare to 76 applicants.<span class="white-space-pre"> </span><a class="app-aware-link " target="_self" href="https://www.linkedin.com/premium/products/?family=JSS&amp;upsellOrderOrigin=premium_job_details_summary_card&amp;utype=job" data-test-app-aware-link="">Try Premium for free</a>'}

time: 5.44 s (started: 2023-03-31 20:15:39 -05:00)


# START SCRAPING

Get all job ids from the job search page into a list

In [13]:
num_page = 0 # to scrape all pages: set to 0 or -1
job_id_list, job_id_scrape_time = get_all_job_ids_from_page(SEARCH_URL, num_page=num_page)
# job_id_list

DATE & TIME: 2023/03/29 13:10:29
Scraping 40 page(s) out of 40 total pages for Data Scientist jobs search in United States...
Found 25 job IDs on page 1. Could not scrape 0 jobs.
Found 25 job IDs on page 2. Could not scrape 0 jobs.
Found 25 job IDs on page 3. Could not scrape 0 jobs.
Found 25 job IDs on page 4. Could not scrape 0 jobs.
Found 25 job IDs on page 5. Could not scrape 0 jobs.
Found 25 job IDs on page 6. Could not scrape 0 jobs.
Found 25 job IDs on page 7. Could not scrape 0 jobs.
Found 25 job IDs on page 8. Could not scrape 0 jobs.
Found 25 job IDs on page 9. Could not scrape 0 jobs.
Found 25 job IDs on page 10. Could not scrape 0 jobs.
Found 25 job IDs on page 11. Could not scrape 0 jobs.
Found 25 job IDs on page 12. Could not scrape 0 jobs.
Found 25 job IDs on page 13. Could not scrape 0 jobs.
Found 25 job IDs on page 14. Could not scrape 0 jobs.
Found 25 job IDs on page 15. Could not scrape 0 jobs.
Found 25 job IDs on page 16. Could not scrape 0 jobs.
Found 25 job IDs on

#### Write list of job ids to file

In [436]:
file_name = f'jobs_{job_id_scrape_time.strftime("%y%m%d_%H%M%S")}.txt'
JOB_FILE_PATH = os.path.join(PROJECT_DIR, ID_DIR_NAME, file_name)
print(f'[{job_id_scrape_time.strftime(r"%Y/%m/%d %H:%M:%S")}] Updated job list at {JOB_FILE_PATH}')

# Write file to folder
# with open(JOB_FILE_PATH, 'w+') as f:
#     for job_id in job_id_list:
#         f.write(f'{job_id}\n')

[2023/03/29 13:10:29] Updated job list at /Users/thule/Desktop/DSProjects/Linkedin_Analysis/job_id_list_dir/jobs_230329_131029.txt


#### Get list of job id from most recent file

In [24]:
# Get most recent id file
JOB_FILE_PATH = os.path.join(PROJECT_DIR, ID_DIR_NAME, (os.listdir(os.path.join(PROJECT_DIR, ID_DIR_NAME)))[-1])

with open(JOB_FILE_PATH, 'r') as f:
    job_id_list = [l.strip() for l in f.readlines()]

job_id_list

885
885
time: 1.85 ms (started: 2023-03-30 09:07:06 -05:00)


Get information from all job page

In [31]:
from tqdm.notebook import tqdm

job_info_list = []

for job_id in tqdm(job_id_list):
    try:
        job_info = get_single_job_info(job_id)
        job_info_list.append(job_info)
    except Exception as e:
        print(e)

job_info_list

  0%|          | 0/485 [00:00<?, ?it/s]

[{'Job ID': '3520044004',
  'Job URL': 'https://www.linkedin.com/jobs/view/3520044004',
  'Name': 'Data Scientist, Research',
  'Company': 'TikTok',
  'Company Logo URL': 'https://media.licdn.com/dms/image/C510BAQGCdThXIss7UQ/company-logo_100_100/0/1539940587971?e=1687996800&v=beta&t=1AyKLKEh0SiRwHFVslSVLJ-5LaOWsCzFqZtQ9ZvHi1o',
  'Location': 'San Jose, CA',
  'Workplace Type': 'Hybrid',
  'Time Posted': '2 weeks ago',
  'Applicants Count': '198 applicants',
  'Job Overview': 'Full-time',
  'Company Overview': '10,001+ employees · Entertainment Providers'},
 {'Job ID': '3523743282',
  'Job URL': 'https://www.linkedin.com/jobs/view/3523743282',
  'Name': 'Global Data Scientist',
  'Company': 'Kimberly-Clark',
  'Company Logo URL': 'https://media.licdn.com/dms/image/C560BAQFahtjOdf_ETQ/company-logo_100_100/0/1542208571146?e=1687996800&v=beta&t=y-aRHU6gnrNyr6nMPswJJHlWEoEOjZyHL-a1Qs5MPFY',
  'Location': 'United States',
  'Workplace Type': 'Remote',
  'Time Posted': '2 weeks ago',
  'Appl

time: 35min 12s (started: 2023-03-30 09:24:02 -05:00)


In [34]:
job_df = pd.DataFrame(job_info_list)
job_df.head(10)

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview
0,3520044004,https://www.linkedin.com/jobs/view/3520044004,"Data Scientist, Research",TikTok,https://media.licdn.com/dms/image/C510BAQGCdThXIss7UQ/company-logo_100_100/0/1539940587971?e=1687996800&v=beta&t=1AyKLKEh0SiRwHFVslSVLJ-5LaOWsCzFqZtQ9ZvHi1o,"San Jose, CA",Hybrid,2 weeks ago,198 applicants,Full-time,"10,001+ employees · Entertainment Providers"
1,3523743282,https://www.linkedin.com/jobs/view/3523743282,Global Data Scientist,Kimberly-Clark,https://media.licdn.com/dms/image/C560BAQFahtjOdf_ETQ/company-logo_100_100/0/1542208571146?e=1687996800&v=beta&t=y-aRHU6gnrNyr6nMPswJJHlWEoEOjZyHL-a1Qs5MPFY,United States,Remote,2 weeks ago,Over 200 applicants,Full-time · Mid-Senior level,"10,001+ employees · Manufacturing"
2,3520459813,https://www.linkedin.com/jobs/view/3520459813,Data Analytics Intern (Summer 2023),Industry Dive,https://media.licdn.com/dms/image/C4E0BAQEAkpLAgFUtpA/company-logo_100_100/0/1520994058492?e=1687996800&v=beta&t=wkHpT8VLjHRZryS_VDdpboOGmMkNRTNhMqVoDrCC2_M,"Washington, DC",Remote,None,None,$16/hr - $21/hr (from job description) · Internship · Internship,201-500 employees · Online Audio and Video Media
3,3525723554,https://www.linkedin.com/jobs/view/3525723554,Data Scientist Solution Specialist- IT Internship,Waters Corporation,https://media.licdn.com/dms/image/C560BAQHFDhBFVWfhzg/company-logo_100_100/0/1656651227123?e=1687996800&v=beta&t=YmLFUIyNIljy5hv0bST7P3wzrzdVBRln9qUpFAufG0c,"Milford, MA",On-site,1 week ago,Over 200 applicants,Internship · Internship,"5,001-10,000 employees · Biotechnology Research"
4,3527821880,https://www.linkedin.com/jobs/view/3527821880,Data Engineer,Chatham Financial,https://media.licdn.com/dms/image/C4D0BAQFPJJtAqKZSKA/company-logo_100_100/0/1566565993951?e=1687996800&v=beta&t=w4VlO0akxyqbbvH7Io6cMb3i8qShWG84zirwSGF-rDM,"Kennett Square, PA",On-site,2 weeks ago,104 applicants,Full-time · Entry level,"501-1,000 employees · Financial Services"
5,3545964641,https://www.linkedin.com/jobs/view/3545964641,"Programmer, Data Analysis, Epidemiology Analytics","Daiichi Sankyo, Inc.",https://media.licdn.com/dms/image/C4E0BAQG95drOHkKpng/company-logo_100_100/0/1598625647052?e=1687996800&v=beta&t=5Udyx0TVHjP69EsvMylTweGLMgYVUfzVxTHcxCxserQ,"Basking Ridge, NJ",None,2 days ago,25 applicants,Full-time,"1,001-5,000 employees · Pharmaceutical Manufacturing"
6,3520729278,https://www.linkedin.com/jobs/view/3520729278,Investment Data Analyst,Westbourne Partners,https://media.licdn.com/dms/image/C560BAQHzeeFsphik2w/company-logo_100_100/0/1644838979087?e=1687996800&v=beta&t=0dJ4YM0X9zk6FQtVPIEz-wM2esA3X0byK0L4Cs-T5Cw,"Chicago, IL",Hybrid,2 weeks ago,Over 200 applicants,"$85,000/yr - $130,000/yr · Full-time · Mid-Senior level",11-50 employees · Staffing and Recruiting
7,3516637194,https://www.linkedin.com/jobs/view/3516637194,"Data Scientist, DC",Rhombus Power Inc.,https://media.licdn.com/dms/image/C560BAQFZ-zC4kM27aQ/company-logo_100_100/0/1626387046517?e=1687996800&v=beta&t=W-n54pZf2YM8Mi-HS2Nq59OUDSpOAebxy-YIJ4I5_H0,"Washington, DC",Hybrid,2 weeks ago,Over 200 applicants,Full-time · Associate,51-200 employees · Defense and Space Manufacturing
8,3527065467,https://www.linkedin.com/jobs/view/3527065467,Data Operations Engineer / Business Operations Analyst,US Tech Solutions,https://media.licdn.com/dms/image/C4D0BAQHPF54GNm3f4w/company-logo_100_100/0/1519856100208?e=1687996800&v=beta&t=RzUD8JFKRffFan_6wyfX_GQZSUbkmy9XbHFIP7PpXoY,"McLean, VA",Hybrid,1 week ago,67 applicants,Full-time · Mid-Senior level,"1,001-5,000 employees · Staffing and Recruiting"
9,3522212905,https://www.linkedin.com/jobs/view/3522212905,Medical Software Quality Engineer - $120k/yr,Bayer,https://media.licdn.com/dms/image/C4E0BAQHKicWqgn2J6g/company-logo_100_100/0/1657870661280?e=1687996800&v=beta&t=WAQmWcZB3S5oRvp4TBIongn5nyCfElrXoGaD2qqUD4M,"Indianola, PA",Hybrid,2 weeks ago,43 applicants,"$120,000/yr · 

time: 8.52 ms (started: 2023-03-30 10:07:05 -05:00)


#### Save DataFrame to csv file

In [43]:
JOB_INFO_PATH = os.path.join(PROJECT_DIR, INFO_DIR_NAME, os.path.basename(JOB_FILE_PATH)[:-3] + 'csv')

job_df.to_csv(JOB_INFO_PATH, index=0, sep='@')

time: 14.4 ms (started: 2023-03-30 23:24:25 -05:00)


#### Load most recent DataFrame from csv file

In [16]:
JOB_INFO_PATH = os.path.join(PROJECT_DIR, INFO_DIR_NAME, (os.listdir(os.path.join(PROJECT_DIR, INFO_DIR_NAME)))[-1])

job_df = pd.read_csv(JOB_INFO_PATH, index_col=0 , sep='@')
job_df.head(10)

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview
0,3520044004,https://www.linkedin.com/jobs/view/3520044004,"Data Scientist, Research",TikTok,https://media.licdn.com/dms/image/C510BAQGCdThXIss7UQ/company-logo_100_100/0/1539940587971?e=1687996800&v=beta&t=1AyKLKEh0SiRwHFVslSVLJ-5LaOWsCzFqZtQ9ZvHi1o,"San Jose, CA",Hybrid,2 weeks ago,198 applicants,Full-time,"10,001+ employees · Entertainment Providers"
1,3523743282,https://www.linkedin.com/jobs/view/3523743282,Global Data Scientist,Kimberly-Clark,https://media.licdn.com/dms/image/C560BAQFahtjOdf_ETQ/company-logo_100_100/0/1542208571146?e=1687996800&v=beta&t=y-aRHU6gnrNyr6nMPswJJHlWEoEOjZyHL-a1Qs5MPFY,United States,Remote,2 weeks ago,Over 200 applicants,Full-time · Mid-Senior level,"10,001+ employees · Manufacturing"
2,3520459813,https://www.linkedin.com/jobs/view/3520459813,Data Analytics Intern (Summer 2023),Industry Dive,https://media.licdn.com/dms/image/C4E0BAQEAkpLAgFUtpA/company-logo_100_100/0/1520994058492?e=1687996800&v=beta&t=wkHpT8VLjHRZryS_VDdpboOGmMkNRTNhMqVoDrCC2_M,"Washington, DC",Remote,NaN,NaN,$16/hr - $21/hr (from job description) · Internship · Internship,201-500 employees · Online Audio and Video Media
3,3525723554,https://www.linkedin.com/jobs/view/3525723554,Data Scientist Solution Specialist- IT Internship,Waters Corporation,https://media.licdn.com/dms/image/C560BAQHFDhBFVWfhzg/company-logo_100_100/0/1656651227123?e=1687996800&v=beta&t=YmLFUIyNIljy5hv0bST7P3wzrzdVBRln9qUpFAufG0c,"Milford, MA",On-site,1 week ago,Over 200 applicants,Internship · Internship,"5,001-10,000 employees · Biotechnology Research"
4,3527821880,https://www.linkedin.com/jobs/view/3527821880,Data Engineer,Chatham Financial,https://media.licdn.com/dms/image/C4D0BAQFPJJtAqKZSKA/company-logo_100_100/0/1566565993951?e=1687996800&v=beta&t=w4VlO0akxyqbbvH7Io6cMb3i8qShWG84zirwSGF-rDM,"Kennett Square, PA",On-site,2 weeks ago,104 applicants,Full-time · Entry level,"501-1,000 employees · Financial Services"
5,3545964641,https://www.linkedin.com/jobs/view/3545964641,"Programmer, Data Analysis, Epidemiology Analytics","Daiichi Sankyo, Inc.",https://media.licdn.com/dms/image/C4E0BAQG95drOHkKpng/company-logo_100_100/0/1598625647052?e=1687996800&v=beta&t=5Udyx0TVHjP69EsvMylTweGLMgYVUfzVxTHcxCxserQ,"Basking Ridge, NJ",NaN,2 days ago,25 applicants,Full-time,"1,001-5,000 employees · Pharmaceutical Manufacturing"
6,3520729278,https://www.linkedin.com/jobs/view/3520729278,Investment Data Analyst,Westbourne Partners,https://media.licdn.com/dms/image/C560BAQHzeeFsphik2w/company-logo_100_100/0/1644838979087?e=1687996800&v=beta&t=0dJ4YM0X9zk6FQtVPIEz-wM2esA3X0byK0L4Cs-T5Cw,"Chicago, IL",Hybrid,2 weeks ago,Over 200 applicants,"$85,000/yr - $130,000/yr · Full-time · Mid-Senior level",11-50 employees · Staffing and Recruiting
7,3516637194,https://www.linkedin.com/jobs/view/3516637194,"Data Scientist, DC",Rhombus Power Inc.,https://media.licdn.com/dms/image/C560BAQFZ-zC4kM27aQ/company-logo_100_100/0/1626387046517?e=1687996800&v=beta&t=W-n54pZf2YM8Mi-HS2Nq59OUDSpOAebxy-YIJ4I5_H0,"Washington, DC",Hybrid,2 weeks ago,Over 200 applicants,Full-time · Associate,51-200 employees · Defense and Space Manufacturing
8,3527065467,https://www.linkedin.com/jobs/view/3527065467,Data Operations Engineer / Business Operations Analyst,US Tech Solutions,https://media.licdn.com/dms/image/C4D0BAQHPF54GNm3f4w/company-logo_100_100/0/1519856100208?e=1687996800&v=beta&t=RzUD8JFKRffFan_6wyfX_GQZSUbkmy9XbHFIP7PpXoY,"McLean, VA",Hybrid,1 week ago,67 applicants,Full-time · Mid-Senior level,"1,001-5,000 employees · Staffing and Recruiting"
9,3522212905,https://www.linkedin.com/jobs/view/3522212905,Medical Software Quality Engineer - $120k/yr,Bayer,https://media.licdn.com/dms/image/C4E0BAQHKicWqgn2J6g/company-logo_100_100/0/1657870661280?e=1687996800&v=beta&t=WAQmWcZB3S5oRvp4TBIongn5nyCfElrXoGaD2qqUD4M,"Indianola, PA",Hybrid,2 weeks ago,43 applicants,"$120,000/yr · Con

time: 12.9 ms (started: 2023-03-31 20:25:26 -05:00)


# CLEANING

In [16]:
job_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656 entries, 0 to 655
Data columns (total 11 columns):
 #   Column            Non-Null Count  Dtype 
---  ------            --------------  ----- 
 0   Job ID            656 non-null    object
 1   Job URL           656 non-null    object
 2   Name              656 non-null    object
 3   Company           655 non-null    object
 4   Company Logo URL  656 non-null    object
 5   Location          656 non-null    object
 6   Workplace Type    561 non-null    object
 7   Time Posted       655 non-null    object
 8   Applicants Count  449 non-null    object
 9   Job Overview      656 non-null    object
 10  Company Overview  656 non-null    object
dtypes: object(11)
memory usage: 56.5+ KB


In [393]:
job_df_2 = job_df.copy() # Make a copy
job_df_2.head()

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview
0,3530951644,https://www.linkedin.com/jobs/view/3530951644,Data Analyst/Administrative Assistant,"Tuba Group, Inc.",https://media.licdn.com/dms/image/C4D0BAQHw44zVuWeBMQ/company-logo_100_100/0/1519951880616?e=1687996800&v=beta&t=9is_bVp6AbucbbnSbDUPHSA6kqgLhpUF7TYuXDA1TsA,"Sioux City, IA",On-site,1 week ago,40 applicants,Contract · Entry level,51-200 employees · Defense and Space Manufacturing
1,3544009597,https://www.linkedin.com/jobs/view/3544009597,Azure Data Engineer,IT Minds LLC,https://media.licdn.com/dms/image/C560BAQEZkh7wG0X0ew/company-logo_100_100/0/1618287716162?e=1687996800&v=beta&t=nLjiBBkms1IPvKEQf2SkcAKcopZV_ISF51Esz0TjGbY,"Charlotte, NC",On-site,3 weeks ago,None,Full-time · Entry level,51-200 employees · IT Services and IT Consulting
2,3541503028,https://www.linkedin.com/jobs/view/3541503028,Big Data Developer,Diverse Lynx,https://media.licdn.com/dms/image/C4E0BAQGk4XHA-0aydA/company-logo_100_100/0/1553721179130?e=1687996800&v=beta&t=iLaN2mgy_v0G0TzRbyYv_laOSXWA7F7pmgATQ0ltneI,"Atlantic City, NJ",Hybrid,3 weeks ago,None,Contract · Entry level,"1,001-5,000 employees · Software Development"
3,3547594589,https://www.linkedin.com/jobs/view/3547594589,Data Scientist @ Remote,Diverse Lynx,https://media.licdn.com/dms/image/C4E0BAQGk4XHA-0aydA/company-logo_100_100/0/1553721179130?e=1687996800&v=beta&t=iLaN2mgy_v0G0TzRbyYv_laOSXWA7F7pmgATQ0ltneI,United States,Remote,27 minutes ago,None,Contract · Entry level,"1,001-5,000 employees · Software Development"
4,3547517166,https://www.linkedin.com/jobs/view/3547517166,Viral Research Scientist I,Compunnel Inc.,https://media.licdn.com/dms/image/C4E0BAQFe-tL0qK8kxw/company-logo_100_100/0/1632140982446?e=1687996800&v=beta&t=2gBvJLsqrfKdM1f8_cG0qi5shKIGVW5As4rdpceMmpo,"Athens, GA",On-site,1 day ago,None,Full-time · Mid-Senior level,"1,001-5,000 employees · IT Services and IT Consulting"


### Split Location
New columns:
- Location_City
- Location_State

In [394]:
job_df_2['Location'].str.split(', ').map(len).value_counts()

2    614
1     34
3      8
Name: Location, dtype: int64

In [395]:
# Remove United States from Location col
job_df_2['Location'] = job_df_2['Location'].str.replace('United States', '', regex=False).str.strip(', ')
# Split city and state from location col
job_df_2[['Location_City', 'Location_State']] = job_df_2['Location'].str.split(', ', expand=True)
job_df_2.head()

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State
0,3530951644,https://www.linkedin.com/jobs/view/3530951644,Data Analyst/Administrative Assistant,"Tuba Group, Inc.",https://media.licdn.com/dms/image/C4D0BAQHw44zVuWeBMQ/company-logo_100_100/0/1519951880616?e=1687996800&v=beta&t=9is_bVp6AbucbbnSbDUPHSA6kqgLhpUF7TYuXDA1TsA,"Sioux City, IA",On-site,1 week ago,40 applicants,Contract · Entry level,51-200 employees · Defense and Space Manufacturing,Sioux City,IA
1,3544009597,https://www.linkedin.com/jobs/view/3544009597,Azure Data Engineer,IT Minds LLC,https://media.licdn.com/dms/image/C560BAQEZkh7wG0X0ew/company-logo_100_100/0/1618287716162?e=1687996800&v=beta&t=nLjiBBkms1IPvKEQf2SkcAKcopZV_ISF51Esz0TjGbY,"Charlotte, NC",On-site,3 weeks ago,None,Full-time · Entry level,51-200 employees · IT Services and IT Consulting,Charlotte,NC
2,3541503028,https://www.linkedin.com/jobs/view/3541503028,Big Data Developer,Diverse Lynx,https://media.licdn.com/dms/image/C4E0BAQGk4XHA-0aydA/company-logo_100_100/0/1553721179130?e=1687996800&v=beta&t=iLaN2mgy_v0G0TzRbyYv_laOSXWA7F7pmgATQ0ltneI,"Atlantic City, NJ",Hybrid,3 weeks ago,None,Contract · Entry level,"1,001-5,000 employees · Software Development",Atlantic City,NJ
3,3547594589,https://www.linkedin.com/jobs/view/3547594589,Data Scientist @ Remote,Diverse Lynx,https://media.licdn.com/dms/image/C4E0BAQGk4XHA-0aydA/company-logo_100_100/0/1553721179130?e=1687996800&v=beta&t=iLaN2mgy_v0G0TzRbyYv_laOSXWA7F7pmgATQ0ltneI,,Remote,27 minutes ago,None,Contract · Entry level,"1,001-5,000 employees · Software Development",,None
4,3547517166,https://www.linkedin.com/jobs/view/3547517166,Viral Research Scientist I,Compunnel Inc.,https://media.licdn.com/dms/image/C4E0BAQFe-tL0qK8kxw/company-logo_100_100/0/1632140982446?e=1687996800&v=beta&t=2gBvJLsqrfKdM1f8_cG0qi5shKIGVW5As4rdpceMmpo,"Athens, GA",On-site,1 day ago,None,Full-time · Mid-Senior level,"1,001-5,000 employees · IT Services and IT Consulting",Athens,GA


### Match State Names with Abbreviation

In [396]:
states_dict = {
'Alabama':	            'AL',
'Kentucky':	            'KY',
'Ohio':	                'OH',
'Alaska':	            'AK',
'Louisiana':	        'LA',
'Oklahoma':	            'OK',
'Arizona':	            'AZ',
'Maine':	            'ME',
'Oregon':	            'OR',
'Arkansas':	            'AR',
'Maryland':	            'MD',
'Pennsylvania':	        'PA',
'American Samoa':	    'AS',
'Massachusetts':	    'MA',
'Puerto Rico':	        'PR',
'California':	        'CA',
'Michigan':	            'MI',
'Rhode Island':	        'RI',
'Colorado':	            'CO',
'Minnesota':	        'MN',
'South Carolina':	    'SC',
'Connecticut':	        'CT',
'Mississippi':	        'MS',
'South Dakota':	        'SD',
'Delaware':	            'DE',
'Missouri':	            'MO',
'Tennessee':	        'TN',
'District of Columbia':	'DC',	
'Montana':	            'MT',
'Texas':	            'TX',
'Florida':	            'FL',
'Nebraska':	            'NE',
'Trust Territories':	'TT',
'Georgia':	            'GA',
'Nevada':	            'NV',
'Utah':	                'UT',
'Guam':	                'GU',
'New Hampshire':	    'NH',
'Vermont':	            'VT',
'Hawaii':	            'HI',
'New Jersey':	        'NJ',
'Virginia':	            'VA',
'Idaho':	            'ID',
'New Mexico':	        'NM',
'Virgin Islands':	    'VI',
'Illinois':	            'IL',
'New York':	            'NY',
'Washington':	        'WA',
'Indiana':	            'IN',
'North Carolina':	    'NC',
'West Virginia':	    'WV',
'Iowa':	                'IA',
'North Dakota':	        'ND',
'Wisconsin':	        'WI',
'Kansas':	            'KS',
'Northern Mariana Islands':	'MP',	
'Wyoming':	            'WY'
}
states_dict = dict(sorted(states_dict.items()))
states_dict = dict((v, k) for k, v in states_dict.items())
states_dict

{'AL': 'Alabama',
 'AK': 'Alaska',
 'AS': 'American Samoa',
 'AZ': 'Arizona',
 'AR': 'Arkansas',
 'CA': 'California',
 'CO': 'Colorado',
 'CT': 'Connecticut',
 'DE': 'Delaware',
 'DC': 'District of Columbia',
 'FL': 'Florida',
 'GA': 'Georgia',
 'GU': 'Guam',
 'HI': 'Hawaii',
 'ID': 'Idaho',
 'IL': 'Illinois',
 'IN': 'Indiana',
 'IA': 'Iowa',
 'KS': 'Kansas',
 'KY': 'Kentucky',
 'LA': 'Louisiana',
 'ME': 'Maine',
 'MD': 'Maryland',
 'MA': 'Massachusetts',
 'MI': 'Michigan',
 'MN': 'Minnesota',
 'MS': 'Mississippi',
 'MO': 'Missouri',
 'MT': 'Montana',
 'NE': 'Nebraska',
 'NV': 'Nevada',
 'NH': 'New Hampshire',
 'NJ': 'New Jersey',
 'NM': 'New Mexico',
 'NY': 'New York',
 'NC': 'North Carolina',
 'ND': 'North Dakota',
 'MP': 'Northern Mariana Islands',
 'OH': 'Ohio',
 'OK': 'Oklahoma',
 'OR': 'Oregon',
 'PA': 'Pennsylvania',
 'PR': 'Puerto Rico',
 'RI': 'Rhode Island',
 'SC': 'South Carolina',
 'SD': 'South Dakota',
 'TN': 'Tennessee',
 'TX': 'Texas',
 'TT': 'Trust Territories',
 'UT': 

In [397]:
job_df_2['Location_State'] = job_df_2['Location_State'].apply(lambda x: states_dict[x] if x in states_dict else None)
job_df_2['Location_State'].unique()

array(['Iowa', 'North Carolina', 'New Jersey', None, 'Georgia',
       'California', 'Indiana', 'Florida', 'Missouri',
       'District of Columbia', 'Maryland', 'Minnesota', 'New York',
       'Michigan', 'Ohio', 'Illinois', 'Alabama', 'Massachusetts',
       'Texas', 'Colorado', 'New Hampshire', 'Pennsylvania', 'Virginia',
       'Kansas', 'Washington', 'Nebraska', 'Connecticut', 'Wyoming',
       'Arizona', 'South Carolina', 'Louisiana', 'Oklahoma', 'Kentucky',
       'Tennessee', 'Wisconsin', 'Alaska', 'North Dakota', 'Nevada',
       'New Mexico', 'Mississippi', 'Rhode Island', 'Utah', 'Oregon',
       'Hawaii', 'Montana', 'Arkansas'], dtype=object)

### Split Company Overview
New columns:
- Company Size
- Industry

In [398]:
job_df_2['Company Overview'].str.split(' · ').map(len).value_counts()

2    650
1      6
Name: Company Overview, dtype: int64

In [399]:
list(set(job_df_2['Company Overview'].str.split(' · ').values.sum()))

['Political Organizations',
 'Administration of Justice',
 'Real Estate',
 'Spectator Sports',
 'Entertainment Providers',
 'Non-profit Organizations',
 'Environmental Services',
 'Defense and Space Manufacturing',
 'Manufacturing',
 'Higher Education',
 '1,001-5,000 employees',
 'Garrett Dobbertin is hiring for this job',
 'Food and Beverage Services',
 'Electric Power Generation',
 'Automotive',
 'Telecommunications',
 '1-10 employees',
 'Industrial Machinery Manufacturing',
 'Retail',
 'Armed Forces',
 'Education Administration Programs',
 'IT Services and IT Consulting',
 '<a class="app-aware-link " target="_self" href="https://www.linkedin.com/search/results/people/?origin=JOB_PAGE_CANNED_SEARCH&amp;currentCompany=%5B166729%5D&amp;schoolFilter=%5B157342%5D" data-test-app-aware-link="">4 school alumni</a>',
 'Venture Capital and Private Equity Principals',
 'Civic and Social Organizations',
 '51-200 employees',
 'Advertising Services',
 'Consumer Goods',
 '201-500 employees',
 'Con

In [400]:
job_df_2.loc[~job_df_2['Company Overview'].str.contains('employees'), 'Company Overview'] = None # replace invalid Company Overview with None
job_df_2[job_df_2['Company Overview'].isna()] # recheck

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State
5,3541502847,https://www.linkedin.com/jobs/view/3541502847,Artificial Intelligence IOS developer,None,"data:image/gif;base64,R0lGODlhAQABAIAAAAAAAP///yH5BAEAAAAALAAAAAABAAEAAAIBRAA7",,Remote,56 minutes ago,1 applicant,Temporary,None,,None
105,3524205334,https://www.linkedin.com/jobs/view/3524205334,Researcher -- E 2023,USG,https://media.licdn.com/dms/image/C4D0BAQEtFVFfSbtRWA/company-logo_100_100/0/1668528042685?e=1687996800&v=beta&t=PP1d3A97jGsesUohpTQHe2XooPDPwEO32l3exXM9828,"Libertyville, IL",On-site,1 week ago,4 applicants,"5,001-10,000 employees · Wholesale Building Materials",None,Libertyville,Illinois
130,3525179725,https://www.linkedin.com/jobs/view/3525179725,AI Research Engineer- Remote,Neo Cybernetica,https://media.licdn.com/dms/image/C560BAQFdt_fuOIJ5TQ/company-logo_100_100/0/1644369586489?e=1687996800&v=beta&t=48gLYFNmehIuoOD3QnQZfOUPPAg-EZncaIXIOVNLs8A,"Bedford, NH",Remote,2 weeks ago,70 applicants,11-50 employees · Software Development,None,Bedford,New Hampshire


In [401]:
job_df_2[job_df_2['Company Overview'].notna()]['Company Overview'].str.split(' · ').map(len).value_counts()

2    650
1      3
Name: Company Overview, dtype: int64

In [402]:
tmp = job_df_2[(job_df_2['Company Overview'].notna())]
tmp[tmp['Company Overview'].str.split('·').map(len)==1]

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State
142,3511519779,https://www.linkedin.com/jobs/view/3511519779,Development Project Manger,Novin Development Corp (NDC),https://media.licdn.com/dms/image/C4D0BAQG0dL80mmXHOw/company-logo_100_100/0/1519906436626?e=1687996800&v=beta&t=Ztw1tnXM0NJh9VxYhYv0RgLAX6aInl0J5E9Ec2wegMc,San Francisco Bay Area,Hybrid,2 weeks ago,Over 200 applicants,"$150,000/yr (from job description) · Full-time",11-50 employees,San Francisco Bay Area,None
513,3501141000,https://www.linkedin.com/jobs/view/3501141000,Communications and Non-Profit Management Internship Summer 2023,Institute for the Study of War,https://media.licdn.com/dms/image/C4D0BAQGQvkGlPGb5Eg/company-logo_100_100/0/1519908135181?e=1687996800&v=beta&t=7rnPeBJpC_x04uzM9IMjrKrMA3mLuK6y4VVhDp76qMo,"Washington, DC",Hybrid,1 month ago,Over 200 applicants,Internship,11-50 employees,Washington,District of Columbia
571,3515314825,https://www.linkedin.com/jobs/view/3515314825,Project Manager - Continuous Improvement,MassMEP,https://media.licdn.com/dms/image/C560BAQHYyK7bdYEOQQ/company-logo_100_100/0/1655235578188?e=1687996800&v=beta&t=Hh_pA2vyR-SmTHl01ihOdUhrbqc5EegSh6FfVp1dlj0,"Auburn, MA",Hybrid,3 weeks ago,158 applicants,"$97,000/yr · Full-time",11-50 employees,Auburn,Massachusetts


In [403]:
# mask = job_df_2[job_df_2['Company Overview'].str.contains('employees')]
job_df_2[['Company Size', 'Industry']] = job_df_2['Company Overview'].str.split(' · ', expand=True)
job_df_2.head()

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State,Company Size,Industry
0,3530951644,https://www.linkedin.com/jobs/view/3530951644,Data Analyst/Administrative Assistant,"Tuba Group, Inc.",https://media.licdn.com/dms/image/C4D0BAQHw44zVuWeBMQ/company-logo_100_100/0/1519951880616?e=1687996800&v=beta&t=9is_bVp6AbucbbnSbDUPHSA6kqgLhpUF7TYuXDA1TsA,"Sioux City, IA",On-site,1 week ago,40 applicants,Contract · Entry level,51-200 employees · Defense and Space Manufacturing,Sioux City,Iowa,51-200 employees,Defense and Space Manufacturing
1,3544009597,https://www.linkedin.com/jobs/view/3544009597,Azure Data Engineer,IT Minds LLC,https://media.licdn.com/dms/image/C560BAQEZkh7wG0X0ew/company-logo_100_100/0/1618287716162?e=1687996800&v=beta&t=nLjiBBkms1IPvKEQf2SkcAKcopZV_ISF51Esz0TjGbY,"Charlotte, NC",On-site,3 weeks ago,None,Full-time · Entry level,51-200 employees · IT Services and IT Consulting,Charlotte,North Carolina,51-200 employees,IT Services and IT Consulting
2,3541503028,https://www.linkedin.com/jobs/view/3541503028,Big Data Developer,Diverse Lynx,https://media.licdn.com/dms/image/C4E0BAQGk4XHA-0aydA/company-logo_100_100/0/1553721179130?e=1687996800&v=beta&t=iLaN2mgy_v0G0TzRbyYv_laOSXWA7F7pmgATQ0ltneI,"Atlantic City, NJ",Hybrid,3 weeks ago,None,Contract · Entry level,"1,001-5,000 employees · Software Development",Atlantic City,New Jersey,"1,001-5,000 employees",Software Development
3,3547594589,https://www.linkedin.com/jobs/view/3547594589,Data Scientist @ Remote,Diverse Lynx,https://media.licdn.com/dms/image/C4E0BAQGk4XHA-0aydA/company-logo_100_100/0/1553721179130?e=1687996800&v=beta&t=iLaN2mgy_v0G0TzRbyYv_laOSXWA7F7pmgATQ0ltneI,,Remote,27 minutes ago,None,Contract · Entry level,"1,001-5,000 employees · Software Development",,None,"1,001-5,000 employees",Software Development
4,3547517166,https://www.linkedin.com/jobs/view/3547517166,Viral Research Scientist I,Compunnel Inc.,https://media.licdn.com/dms/image/C4E0BAQFe-tL0qK8kxw/company-logo_100_100/0/1632140982446?e=1687996800&v=beta&t=2gBvJLsqrfKdM1f8_cG0qi5shKIGVW5As4rdpceMmpo,"Athens, GA",On-site,1 day ago,None,Full-time · Mid-Senior level,"1,001-5,000 employees · IT Services and IT Consulting",Athens,Georgia,"1,001-5,000 employees",IT Services and IT Consulting


only works when comp size is avail

### Create Salary Column

In [404]:
# salary_pattern = r"\$(\d)+.*\/(yr|hr)"
# seq = '$50,000/yr - $75,000/yr (from job description) · Full-time · Associate'
# seq = '$16.74/hr - $21.75/hr'
# seq = '$20/hr - $23/hr'
# seq = '$40/hr - $45/hr'
# result = re.search(salary_pattern, seq)
# result.group()

In [405]:
salary_pattern = r"\$(\d)+.*\/(yr|hr)"

for idx, row in job_df_2.iterrows():
    job_overview = row['Job Overview']
    try:
        salary = re.search(salary_pattern, job_overview)
        job_df_2.at[idx, 'Salary'] = salary.group() if salary is not None else None
    except NoSuchElementException:
        pass

job_df_2.head(5)

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State,Company Size,Industry,Salary
0,3530951644,https://www.linkedin.com/jobs/view/3530951644,Data Analyst/Administrative Assistant,"Tuba Group, Inc.",https://media.licdn.com/dms/image/C4D0BAQHw44zVuWeBMQ/company-logo_100_100/0/1519951880616?e=1687996800&v=beta&t=9is_bVp6AbucbbnSbDUPHSA6kqgLhpUF7TYuXDA1TsA,"Sioux City, IA",On-site,1 week ago,40 applicants,Contract · Entry level,51-200 employees · Defense and Space Manufacturing,Sioux City,Iowa,51-200 employees,Defense and Space Manufacturing,NaN
1,3544009597,https://www.linkedin.com/jobs/view/3544009597,Azure Data Engineer,IT Minds LLC,https://media.licdn.com/dms/image/C560BAQEZkh7wG0X0ew/company-logo_100_100/0/1618287716162?e=1687996800&v=beta&t=nLjiBBkms1IPvKEQf2SkcAKcopZV_ISF51Esz0TjGbY,"Charlotte, NC",On-site,3 weeks ago,None,Full-time · Entry level,51-200 employees · IT Services and IT Consulting,Charlotte,North Carolina,51-200 employees,IT Services and IT Consulting,NaN
2,3541503028,https://www.linkedin.com/jobs/view/3541503028,Big Data Developer,Diverse Lynx,https://media.licdn.com/dms/image/C4E0BAQGk4XHA-0aydA/company-logo_100_100/0/1553721179130?e=1687996800&v=beta&t=iLaN2mgy_v0G0TzRbyYv_laOSXWA7F7pmgATQ0ltneI,"Atlantic City, NJ",Hybrid,3 weeks ago,None,Contract · Entry level,"1,001-5,000 employees · Software Development",Atlantic City,New Jersey,"1,001-5,000 employees",Software Development,NaN
3,3547594589,https://www.linkedin.com/jobs/view/3547594589,Data Scientist @ Remote,Diverse Lynx,https://media.licdn.com/dms/image/C4E0BAQGk4XHA-0aydA/company-logo_100_100/0/1553721179130?e=1687996800&v=beta&t=iLaN2mgy_v0G0TzRbyYv_laOSXWA7F7pmgATQ0ltneI,,Remote,27 minutes ago,None,Contract · Entry level,"1,001-5,000 employees · Software Development",,None,"1,001-5,000 employees",Software Development,NaN
4,3547517166,https://www.linkedin.com/jobs/view/3547517166,Viral Research Scientist I,Compunnel Inc.,https://media.licdn.com/dms/image/C4E0BAQFe-tL0qK8kxw/company-logo_100_100/0/1632140982446?e=1687996800&v=beta&t=2gBvJLsqrfKdM1f8_cG0qi5shKIGVW5As4rdpceMmpo,"Athens, GA",On-site,1 day ago,None,Full-time · Mid-Senior level,"1,001-5,000 employees · IT Services and IT Consulting",Athens,Georgia,"1,001-5,000 employees",IT Services and IT Consulting,NaN


### Split Job Overview

- Workplace Type
- Level of Expertise

In [406]:
tmp = job_df_2.copy()
tmp = tmp['Job Overview'].str.replace(' (from job description)', '', regex=False)
tmp = tmp.str.replace(r'\$(\d)+.*\/(yr|hr)', '', regex=True).str.strip(' · ')
list(set(tmp.str.split(' · ').values.sum()))

['11-50 employees',
 '5,001-10,000 employees',
 'Director',
 'Contract',
 'Wholesale Building Materials',
 'Entry level',
 'Temporary',
 'Full-time',
 'Software Development',
 'Executive',
 'Associate',
 'Part-time',
 'Internship',
 'Mid-Senior level']

In [407]:
contract_type_list = ['Temporary', 'Part-time', 'Full-time', 'Internship', 'Contract']
contract_type_pattern = '|'.join(contract_type_list)

for idx, row in job_df_2.iterrows():
    job_overview = row['Job Overview']
    try:
        contract_type = re.search(contract_type_pattern, job_overview)
        job_df_2.at[idx, 'Contract Type'] = contract_type.group() if contract_type is not None else None
    except NoSuchElementException:
        pass
job_df_2.head(5)

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State,Company Size,Industry,Salary,Contract Type
0,3530951644,https://www.linkedin.com/jobs/view/3530951644,Data Analyst/Administrative Assistant,"Tuba Group, Inc.",https://media.licdn.com/dms/image/C4D0BAQHw44zVuWeBMQ/company-logo_100_100/0/1519951880616?e=1687996800&v=beta&t=9is_bVp6AbucbbnSbDUPHSA6kqgLhpUF7TYuXDA1TsA,"Sioux City, IA",On-site,1 week ago,40 applicants,Contract · Entry level,51-200 employees · Defense and Space Manufacturing,Sioux City,Iowa,51-200 employees,Defense and Space Manufacturing,NaN,Contract
1,3544009597,https://www.linkedin.com/jobs/view/3544009597,Azure Data Engineer,IT Minds LLC,https://media.licdn.com/dms/image/C560BAQEZkh7wG0X0ew/company-logo_100_100/0/1618287716162?e=1687996800&v=beta&t=nLjiBBkms1IPvKEQf2SkcAKcopZV_ISF51Esz0TjGbY,"Charlotte, NC",On-site,3 weeks ago,None,Full-time · Entry level,51-200 employees · IT Services and IT Consulting,Charlotte,North Carolina,51-200 employees,IT Services and IT Consulting,NaN,Full-time
2,3541503028,https://www.linkedin.com/jobs/view/3541503028,Big Data Developer,Diverse Lynx,https://media.licdn.com/dms/image/C4E0BAQGk4XHA-0aydA/company-logo_100_100/0/1553721179130?e=1687996800&v=beta&t=iLaN2mgy_v0G0TzRbyYv_laOSXWA7F7pmgATQ0ltneI,"Atlantic City, NJ",Hybrid,3 weeks ago,None,Contract · Entry level,"1,001-5,000 employees · Software Development",Atlantic City,New Jersey,"1,001-5,000 employees",Software Development,NaN,Contract
3,3547594589,https://www.linkedin.com/jobs/view/3547594589,Data Scientist @ Remote,Diverse Lynx,https://media.licdn.com/dms/image/C4E0BAQGk4XHA-0aydA/company-logo_100_100/0/1553721179130?e=1687996800&v=beta&t=iLaN2mgy_v0G0TzRbyYv_laOSXWA7F7pmgATQ0ltneI,,Remote,27 minutes ago,None,Contract · Entry level,"1,001-5,000 employees · Software Development",,None,"1,001-5,000 employees",Software Development,NaN,Contract
4,3547517166,https://www.linkedin.com/jobs/view/3547517166,Viral Research Scientist I,Compunnel Inc.,https://media.licdn.com/dms/image/C4E0BAQFe-tL0qK8kxw/company-logo_100_100/0/1632140982446?e=1687996800&v=beta&t=2gBvJLsqrfKdM1f8_cG0qi5shKIGVW5As4rdpceMmpo,"Athens, GA",On-site,1 day ago,None,Full-time · Mid-Senior level,"1,001-5,000 employees · IT Services and IT Consulting",Athens,Georgia,"1,001-5,000 employees",IT Services and IT Consulting,NaN,Full-time


In [408]:
exp_levels_list = ['Entry level', 'Junior', 'Mid-Senior level', 'Associate', 'Executive', 'Director']
exp_levels_pattern = '|'.join(exp_levels_list)

for idx, row in job_df_2.iterrows():
    job_overview = row['Job Overview']
    try:
        exp_level = re.search(exp_levels_pattern, job_overview)
        job_df_2.at[idx, 'Level of Expertise'] = exp_level.group() if exp_level is not None else None
    except NoSuchElementException:
        pass
job_df_2.head(5)

,Job ID,Job URL,Name,Company,Company Logo URL,Location,Workplace Type,Time Posted,Applicants Count,Job Overview,Company Overview,Location_City,Location_State,Company Size,Industry,Salary,Contract Type,Level of Expertise
0,3530951644,https://www.linkedin.com/jobs/view/3530951644,Data Analyst/Administrative Assistant,"Tuba Group, Inc.",https://media.licdn.com/dms/image/C4D0BAQHw44zVuWeBMQ/company-logo_100_100/0/1519951880616?e=1687996800&v=beta&t=9is_bVp6AbucbbnSbDUPHSA6kqgLhpUF7TYuXDA1TsA,"Sioux City, IA",On-site,1 week ago,40 applicants,Contract · Entry level,51-200 employees · Defense and Space Manufacturing,Sioux City,Iowa,51-200 employees,Defense and Space Manufacturing,NaN,Contract,Entry level
1,3544009597,https://www.linkedin.com/jobs/view/3544009597,Azure Data Engineer,IT Minds LLC,https://media.licdn.com/dms/image/C560BAQEZkh7wG0X0ew/company-logo_100_100/0/1618287716162?e=1687996800&v=beta&t=nLjiBBkms1IPvKEQf2SkcAKcopZV_ISF51Esz0TjGbY,"Charlotte, NC",On-site,3 weeks ago,None,Full-time · Entry level,51-200 employees · IT Services and IT Consulting,Charlotte,North Carolina,51-200 employees,IT Services and IT Consulting,NaN,Full-time,Entry level
2,3541503028,https://www.linkedin.com/jobs/view/3541503028,Big Data Developer,Diverse Lynx,https://media.licdn.com/dms/image/C4E0BAQGk4XHA-0aydA/company-logo_100_100/0/1553721179130?e=1687996800&v=beta&t=iLaN2mgy_v0G0TzRbyYv_laOSXWA7F7pmgATQ0ltneI,"Atlantic City, NJ",Hybrid,3 weeks ago,None,Contract · Entry level,"1,001-5,000 employees · Software Development",Atlantic City,New Jersey,"1,001-5,000 employees",Software Development,NaN,Contract,Entry level
3,3547594589,https://www.linkedin.com/jobs/view/3547594589,Data Scientist @ Remote,Diverse Lynx,https://media.licdn.com/dms/image/C4E0BAQGk4XHA-0aydA/company-logo_100_100/0/1553721179130?e=1687996800&v=beta&t=iLaN2mgy_v0G0TzRbyYv_laOSXWA7F7pmgATQ0ltneI,,Remote,27 minutes ago,None,Contract · Entry level,"1,001-5,000 employees · Software Development",,None,"1,001-5,000 employees",Software Development,NaN,Contract,Entry level
4,3547517166,https://www.linkedin.com/jobs/view/3547517166,Viral Research Scientist I,Compunnel Inc.,https://media.licdn.com/dms/image/C4E0BAQFe-tL0qK8kxw/company-logo_100_100/0/1632140982446?e=1687996800&v=beta&t=2gBvJLsqrfKdM1f8_cG0qi5shKIGVW5As4rdpceMmpo,"Athens, GA",On-site,1 day ago,None,Full-time · Mid-Senior level,"1,001-5,000 employees · IT Services and IT Consulting",Athens,Georgia,"1,001-5,000 employees",IT Services and IT Consulting,NaN,Full-time,Mid-Senior level


### Replace all None and emptry strings with NaN

In [430]:
job_df_2 = job_df_2.replace('', np.nan).fillna('Unknown')

29/03/23 Update:
- Fix scraping Job & Company Overview 

# EDA

In [427]:
cols = ['Job URL', 'Name', 'Company', 'Location_City', 'Location_State',
       'Workplace Type', 'Time Posted', 'Applicants Count',
        'Salary', 'Contract Type', 'Level of Expertise',
       'Company Size', 'Industry']
job_df_2 = job_df_2[cols]

In [431]:
job_df_2.head(10)

,Job URL,Name,Company,Location_City,Location_State,Workplace Type,Time Posted,Applicants Count,Salary,Contract Type,Level of Expertise,Company Size,Industry
0,https://www.linkedin.com/jobs/view/3530951644,Data Analyst/Administrative Assistant,"Tuba Group, Inc.",Sioux City,Iowa,On-site,1 week ago,40 applicants,Unknown,Contract,Entry level,51-200 employees,Defense and Space Manufacturing
1,https://www.linkedin.com/jobs/view/3544009597,Azure Data Engineer,IT Minds LLC,Charlotte,North Carolina,On-site,3 weeks ago,Unknown,Unknown,Full-time,Entry level,51-200 employees,IT Services and IT Consulting
2,https://www.linkedin.com/jobs/view/3541503028,Big Data Developer,Diverse Lynx,Atlantic City,New Jersey,Hybrid,3 weeks ago,Unknown,Unknown,Contract,Entry level,"1,001-5,000 employees",Software Development
3,https://www.linkedin.com/jobs/view/3547594589,Data Scientist @ Remote,Diverse Lynx,Unknown,Unknown,Remote,27 minutes ago,Unknown,Unknown,Contract,Entry level,"1,001-5,000 employees",Software Development
4,https://www.linkedin.com/jobs/view/3547517166,Viral Research Scientist I,Compunnel Inc.,Athens,Georgia,On-site,1 day ago,Unknown,Unknown,Full-time,Mid-Senior level,"1,001-5,000 employees",IT Services and IT Consulting
5,https://www.linkedin.com/jobs/view/3541502847,Artificial Intelligence IOS developer,Unknown,Unknown,Unknown,Remote,56 minutes ago,1 applicant,Unknown,Temporary,Unknown,Unknown,Unknown
6,https://www.linkedin.com/jobs/view/3526821605,Data Scientist,Diverse Lynx,Santa Clara,California,Hybrid,1 week ago,11 applicants,Unknown,Contract,Entry level,"1,001-5,000 employees",Software Development
7,https://www.linkedin.com/jobs/view/3513213441,Dev10 Entry Level Data Engineer - Nationwide,Genesis10,Fort Wayne,Indiana,On-site,2 weeks ago,5 applicants,Unknown,Full-time,Entry level,"1,001-5,000 employees",IT Services and IT Consulting
8,https://www.linkedin.com/jobs/view/3532768586,68053338 - DATA BASE ANALYST,State of Florida,Tallahassee,Florida,On-site,1 week ago,4 applicants,Unknown,Part-time,Entry level,"10,001+ employees",Government Administration
9,https://www.linkedin.com/jobs/view/3529968990,Sr Project Manager - Corporate Talent Acquisition,H&amp;R Block,Kansas City,Missouri,On-site,1 week ago,25 applicants,Unknown,Full-time,Mid-Senior level,"10,001+ employees",Retail


In [432]:
job_df_2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 656 entries, 0 to 655
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype 
---  ------              --------------  ----- 
 0   Job URL             656 non-null    object
 1   Name                656 non-null    object
 2   Company             656 non-null    object
 3   Location_City       656 non-null    object
 4   Location_State      656 non-null    object
 5   Workplace Type      656 non-null    object
 6   Time Posted         656 non-null    object
 7   Applicants Count    656 non-null    object
 8   Salary              656 non-null    object
 9   Contract Type       656 non-null    object
 10  Level of Expertise  656 non-null    object
 11  Company Size        656 non-null    object
 12  Industry            656 non-null    object
dtypes: object(13)
memory usage: 66.8+ KB


In [433]:
job_df_2['Location_State'].value_counts()

California              89
Unknown                 58
Texas                   53
New York                44
Illinois                36
New Jersey              34
Virginia                34
Maryland                31
Massachusetts           24
Florida                 21
Ohio                    21
North Carolina          20
Colorado                19
Michigan                16
Minnesota               14
District of Columbia    13
Pennsylvania            11
Alabama                 11
Georgia                 11
Washington              11
Missouri                 9
Arizona                  8
Connecticut              6
Indiana                  6
Nebraska                 5
Iowa                     5
New Hampshire            5
Kentucky                 4
Tennessee                4
Utah                     3
Rhode Island             3
Wisconsin                3
Kansas                   3
Oklahoma                 3
Nevada                   2
New Mexico               2
Mississippi              2
S

In [434]:
job_df_2['Industry'].value_counts()

IT Services and IT Consulting              134
Business Consulting and Services            94
Staffing and Recruiting                     90
Software Development                        47
Hospitals and Health Care                   40
                                          ... 
Computers and Electronics Manufacturing      1
Computer Hardware                            1
Chemical Manufacturing                       1
Spectator Sports                             1
International Trade and Development          1
Name: Industry, Length: 82, dtype: int64

### Entry-Level Job

In [439]:
entry_jobs_df = job_df_2[job_df_2['Level of Expertise']=='Entry level']
entry_jobs_df.head()

,Job URL,Name,Company,Location_City,Location_State,Workplace Type,Time Posted,Applicants Count,Salary,Contract Type,Level of Expertise,Company Size,Industry
0,https://www.linkedin.com/jobs/view/3530951644,Data Analyst/Administrative Assistant,"Tuba Group, Inc.",Sioux City,Iowa,On-site,1 week ago,40 applicants,Unknown,Contract,Entry level,51-200 employees,Defense and Space Manufacturing
1,https://www.linkedin.com/jobs/view/3544009597,Azure Data Engineer,IT Minds LLC,Charlotte,North Carolina,On-site,3 weeks ago,Unknown,Unknown,Full-time,Entry level,51-200 employees,IT Services and IT Consulting
2,https://www.linkedin.com/jobs/view/3541503028,Big Data Developer,Diverse Lynx,Atlantic City,New Jersey,Hybrid,3 weeks ago,Unknown,Unknown,Contract,Entry level,"1,001-5,000 employees",Software Development
3,https://www.linkedin.com/jobs/view/3547594589,Data Scientist @ Remote,Diverse Lynx,Unknown,Unknown,Remote,27 minutes ago,Unknown,Unknown,Contract,Entry level,"1,001-5,000 employees",Software Development
6,https://www.linkedin.com/jobs/view/3526821605,Data Scientist,Diverse Lynx,Santa Clara,California,Hybrid,1 week ago,11 applicants,Unknown,Contract,Entry level,"1,001-5,000 employees",Software Development
